<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/main/workflows/TotalSegmentator/Notebooks/downloadDicomAndConvertNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This Notebook can download CT data from Imaging Data Commons and convert to NIfTI with dcm2niix**

<img src="https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/Docs/images/download_convert.png">

DICOM files are downloaded from IDC and converted to NIFTI files with dcm2niix. Whenever a non CT series or if there are multiple NIFTI files generated for a series, such series are prohibited from continuing to Inference. A CSV file is created with a list of such series.

Please cite:

Li X, Morgan PS, Ashburner J, Smith J, Rorden C. (2016) The first step for neuroimaging data analysis: DICOM to NIfTI conversion. J Neurosci Methods. 264:47-56.

Fedorov A, Longabaugh WJR, Pot D, Clunie DA, Pieper SD, Gibbs DL, Bridge C, Herrmann MD, Homeyer A, Lewis R, Aerts HJWL, Krishnaswamy D, Thiriveedhi VK, Ciausu C, Schacherer DP, Bontempi D, Pihl T, Wagner U, Farahani K, Kim E, Kikinis R. National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence. Radiographics. 2023 Dec;43(12):e230180. doi: 10.1148/rg.230180. PMID: 37999984; PMCID: PMC10716669.


## **Ways to utilize this notebook**


*   **Colab**
*   **DockerContainer/Terra/SB-CGC**


#### **Colab**
*  This notebook was initally developed and tested on Colab, and a working version is saved on github, however reproducibility may not be guaranteed as the run time environment changes with colab updates
*  To run this notebook with Colab, Click 'Open In Colab' icon on top left
*  In 'interactive' mode, a list of seriesInstanceUIDs are chosen by default but a user may modify them to their use case
*  Run each cell to install the packages and to download the data from IDC, convert to NIfTI saved in lz4 compressed format

#### **Docker**
* This notebook is primarly developed to be used on Terra/SB-CGC platforms using Docker
* Running this notebook in a docker container ensures reproduciblity, as we lock the run environment beginning from the base docker image to pip packages in the docker image
* Docker images can be found @ https://hub.docker.com/repository/docker/imagingdatacommons/download_convert/tags
* The link to dockerfile along with git commit hash used for building the docker image can be found in one of the layers called 'LABEL'

    <img src="https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/Docs/images/download_convert_docker.png">

* We use a python package called Papermill, that can run the notebook with out having to convert it to a python script. This allows us maintain one copy of code instead of two.
* A sample papermill command is
    <pre>
    papermill downloadDicomAndConvertNotebook.ipynb outputdownloadDicomAndConvertNotebook.ipynb -y SeriesInstanceUIDs yamlListOfSeriesInstanceUIDs
    </pre>
    * refer to https://papermill.readthedocs.io/en/latest/usage-execute.html#note-about-using-yaml on how to pass the yaml list
    
    Instead of passing the yaml list as a string, a yaml file can also be passed, as we do for SB-CGC
    [an example can be found here](!https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/Docs/sampleManifests/batch_1.yaml)
    <pre>
    papermill downloadDicomAndConvertNotebook.ipynb outputdownloadDicomAndConvertNotebook.ipynb -f path_to_yamlListOfSeriesInstanceUIDs.yaml
    </pre>


### **Installing Packages**

In [ ]:
%%capture
import sys
if 'google.colab' in sys.modules:
  #Install apt packages
  !apt-get update \
    && apt-get install -y --no-install-recommends \
      dcm2niix\
      lz4\
      pigz\
      #plastimatch\
      wget\
      zip\
    && rm -rf /var/lib/apt/lists/*

In [ ]:
%%capture
if 'google.colab' in sys.modules:
  !pip install idc-index==0.2.8

In [ ]:
%%capture
if 'google.colab' in sys.modules:
  #install s5cmd
  !wget "https://github.com/peak/s5cmd/releases/download/v2.2.2/s5cmd_2.2.2_Linux-64bit.tar.gz"\
    && tar -xvzf "s5cmd_2.2.2_Linux-64bit.tar.gz"\
    && rm "s5cmd_2.2.2_Linux-64bit.tar.gz"\
    && mv s5cmd /usr/local/bin/s5cmd

### **Importing Packages**

In [ ]:
import csv
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from idc_index import index
from pathlib import Path
from time import sleep
import glob
import matplotlib.pyplot as plt
import os
import pandas as pd
import psutil
import shutil
import subprocess
import sys
import time


### **Current Environment**

In [ ]:
curr_dir   = Path().absolute()

print(time.asctime(time.localtime()))
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

### **Initialize IDC Client**

we use idc-index pypi package to handle downloading data from IDC. 
In this notebook, we are using version 0.2.8 which contains the index from idc version 17

Learn more about idc-index at https://github.com/ImagingDataCommons/idc-index

In [ ]:
idc_client=index.IDCClient()

### **For local testing**

By default a list of seriesInstanceUIDs are chosen  here. However, you can modify them to your usecase.

Below cell is also tagged as `parameters`, so that when running this notebook in non interactive mode on Terra or Seven Bridges Genomics- Cancer Genomics Cloud platforms, papermill will inject a cell to pass the yaml list of SeriesInstanceUIDs

In [ ]:
if 'google.colab' in sys.modules:
  SeriesInstanceUIDs=['1.3.6.1.4.1.14519.5.2.1.7009.9004.100143549999116733615345241533',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100148350742920339334061834697',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100241427395754063917290539621',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100266844261017577841946689119',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100554983367710060268444607770',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100609092457306482866656779396',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100766949446324115207263771273',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100807683971079396484581858934',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100898243941555041874482639283',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100962349324934756610763981593',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100993923831797845405183790593',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.101126369092366339550409924127']


### **Defining Functions**

In [ ]:
#Creating Directories
try:
  shutil.rmtree('dcm2niix')
except OSError:
  pass
os.mkdir('dcm2niix')

In [ ]:
def download_dicom_data(series_id: str) -> None:
    """
    Downloads raw DICOM data

    Args:
    series_id: The DICOM Tag SeriesInstanceUID of the DICOM series to be converted.
    """
    download_directory = f"idc_data/{series_id}"
    # Attempt to remove the directory for the series if it exists
    try:
        shutil.rmtree(download_directory)
    except OSError:
        pass
    print(f'\n Downloading DICOM files from IDC Storage Buckets \n')
    idc_client.download_dicom_series(seriesInstanceUID= series_id, downloadDir=download_directory, quiet=False)



In [ ]:
def is_series_CT(series_id: str) -> bool:
    """
    Gets the image modality for the corresponding seriesInstanceUID from idc-index
    Refer to this query for additional columns available in idc-index!

    https://github.com/ImagingDataCommons/idc-index/blob/main/queries/idc_index.sql

    Args:
    series_id: The DICOM Tag SeriesInstanceUID of the DICOM series to be processed.
    """

    query = f"""
    SELECT
    Modality
    FROM index
    WHERE SeriesInstanceUID = '{series_id}'
    """

    try:
        modality_df = idc_client.sql_query(query)
        if not modality_df.empty:
            modality = modality_df['Modality'][0]
            if modality=='CT':
              return True
            else:
              log_modality_errors(series_id)
              return False
        else:
            log_modality_errors(series_id)
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        log_modality_errors(series_id)
        return False


In [ ]:
def log_modality_errors(series_id: str) -> None:
    """
    Logs an error when the modality is not CT for a given series.

    Args:
        series_id: The ID of the series.
    """
    # Open the log file in append mode
    with open("modality_error_file.txt", "a") as f:
        # Write the error message to the file
        f.write(f"Error: Modality is not CT for series {series_id}\n")


In [ ]:
def convert_dicom_to_nifti(series_id: str) -> None:
    """
    Converts a DICOM series to a NIfTI file.

    Args:
      series_id: The DICOM Tag SeriesInstanceUID of the DICOM series to be converted.
    """

    # Attempt to remove the directory for the series if it exists
    try:
        shutil.rmtree(f"dcm2niix/{series_id}")
    except OSError:
        pass

    # Create a new directory for the series
    os.mkdir(f"dcm2niix/{series_id}")

    print("\n Converting DICOM files to NIfTI \n")

    # Run the appropriate converter command and capture the output

    result = subprocess.run(
        f"dcm2niix -z y -f %j_%p_%t_%s -b n -m y -o dcm2niix/{series_id} idc_data/{series_id}",
        shell=True,
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    print("\n Conversion successful \n")

In [ ]:
def download_and_process_series(series_id: str, runtime_stats: pd.DataFrame) -> pd.DataFrame:
  """Downloads and processes a DICOM series.

  Args:
    series_id: The identifier of the DICOM series to be processed.
    runtime_stats: DataFrame to store runtime statistics.

  Returns:
    Updated DataFrame with runtime statistics.
  """

  # Create a DataFrame to track the processing times.
  log = pd.DataFrame({'SeriesInstanceUID': [series_id]})

  print(f"\n Processing series: {series_id}")

  if is_series_CT(series_id):

      # Start the timer for downloading the DICOM series.
      start_time = time.time()
      download_dicom_data(series_id)
      download_time = time.time() - start_time

      print(f'Downloaded in %g seconds'% download_time)

      # Add the download time to the DataFrame.
      log['download_time'] = download_time

      # Start the timer for converting the DICOM series to NIfTI.
      start_time = time.time()
      convert_dicom_to_nifti(series_id)
      convert_dicom_to_nifti_time = time.time() - start_time

      # Add the conversion time to the DataFrame.
      log['NiftiConverter_time'] = convert_dicom_to_nifti_time

  else:
     log['download_time'] = 0
     log['NiftiConverter_time'] = 0

  # Update the runtime statistics DataFrame.
  runtime_stats = pd.concat([runtime_stats, log], ignore_index=True, axis=0)

  return runtime_stats

In [ ]:
class MemoryMonitor:
    def __init__(self):
        # Flag to control the measurement loop
        self.keep_measuring = True
        # Get the path of the working disk
        self.working_disk_path = self.get_working_disk_path()

    def get_working_disk_path(self):
        # This code is specific to Terra/SB-CGC as multiple disks are mounted on the platforms

        # Get all disk partitions
        partitions = psutil.disk_partitions()
        for partition in partitions:
            # If root partition, return root path
            if partition.mountpoint == '/':
                return '/'
            # If cromwell_root is in mountpoint, return cromwell_root path
            elif '/cromwell_root' in partition.mountpoint:
                return '/cromwell_root'
        # Default to root directory if no specific path is found
        return '/'

    def measure_usage(self):
        # Initialize lists to store measurements
        cpu_usage = []
        ram_usage_mb = []
        disk_usage_all = []
        time_stamps = []

        # Record start time
        start_time = time.time()

        while self.keep_measuring:
            # Measure CPU usage
            cpu = psutil.cpu_percent()

            # Measure RAM usage
            ram = psutil.virtual_memory()

            # Measure disk usage
            disk_usage = psutil.disk_usage(self.working_disk_path)

            # Calculate used and total disk space in GB
            disk_used = disk_usage.used / 1000 / 1000 / 1000
            disk_total = disk_usage.total / 1000 / 1000 / 1000

            # Calculate total and used RAM in MB
            ram_total_mb = ram.total / 1000 / 1000
            ram_mb = (ram.total - ram.available) / 1000 / 1000

            # Append measurements to lists
            cpu_usage.append(cpu)
            ram_usage_mb.append(ram_mb)
            disk_usage_all.append(disk_used)

            # Record timestamp relative to start time
            time_stamps.append(time.time() - start_time)

            # Wait for a second before next measurement
            sleep(1)

        # Return all measurements and totals
        return cpu_usage, ram_usage_mb, time_stamps, ram_total_mb, disk_usage_all, disk_total


### **Downloading and Converting**

In [ ]:
# Initialize a DataFrame to store runtime statistics
runtime_stats = pd.DataFrame(columns=['SeriesInstanceUID','download_time',
                                      'NiftiConverter_time', 'cpu_usage','ram_usage_mb', 'ram_total_mb', 'disk_usage_all', 'disk_total'
                                      ])

if __name__ == "__main__":
    # Loop over all series IDs
    for series_id in SeriesInstanceUIDs:
        # Create a ThreadPoolExecutor
        with ThreadPoolExecutor() as executor:
            # Initialize a MemoryMonitor instance
            monitor = MemoryMonitor()
            # Start a new thread to measure memory usage
            mem_thread = executor.submit(monitor.measure_usage)
            try:
                # Start a new thread to download and process the series
                proc_thread = executor.submit(download_and_process_series, series_id, runtime_stats)
                # Wait for the processing thread to finish
                runtime_stats = proc_thread.result()
            finally:
                # Stop the memory monitor thread
                monitor.keep_measuring = False
                # Get the results from the memory monitor thread
                cpu_usage, ram_usage_mb, time_stamps, ram_total_mb, disk_usage_all, disk_total= mem_thread.result()

                # Update the runtime statistics DataFrame with the results
                cpu_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[cpu_idx, runtime_stats.columns.get_loc('cpu_usage')] = [[cpu_usage]]

                ram_usage_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[ram_usage_mb_idx, runtime_stats.columns.get_loc('ram_usage_mb')] = [[ram_usage_mb]]

                ram_total_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[ram_total_mb_idx, runtime_stats.columns.get_loc('ram_total_mb')] = [[ram_total_mb]]

                disk_usage_gb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[disk_usage_gb_idx, runtime_stats.columns.get_loc('disk_usage_all')] = [[disk_usage_all]]

                # Update total disk space for all rows (assuming it's the same for all series)
                runtime_stats['disk_total']=disk_total

                # Plot CPU usage, memory usage and disk usage over time
                fig, ((ax1,ax2, ax3)) = plt.subplots(1,3, figsize=(12, 4))

                ax1.plot(time_stamps, cpu_usage)
                ax1.set_ylim(0, 100)
                ax1.set_xlabel('Time (s)')
                ax1.set_ylabel('CPU usage (%)')

                ax2.plot(time_stamps, ram_usage_mb)
                ax2.set_ylim(0, ram_total_mb)
                ax2.set_xlabel('Time (s)')
                ax2.set_ylabel('Memory usage (MB)')

                ax3.plot(time_stamps, disk_usage_all)
                ax3.set_ylim(0, disk_total)
                ax3.set_xlabel('Time (s)')
                ax3.set_ylabel('Disk usage (GB)')

                plt.show()


### **Monitoring for dcm2niix Errors**

In [ ]:
def check_dcm2niix_errors(path: str) -> None:
    """
    Check for errors in the conversion of DICOM to NIfTI files.

    Args:
    path: The path to the directory containing the series directories.
    """
    # Loop over all series directories in the path
    for series_id in os.listdir(path):
        series_id_path = os.path.join(path, series_id)

        # Check if the path is a directory
        if os.path.isdir(series_id_path):
            # Count the number of files in the directory
            num_files = len([f for f in os.listdir(series_id_path) if os.path.isfile(os.path.join(series_id_path, f))])

            # If no files or more than one file found, log an error and remove the directory
            if num_files == 0 or num_files > 1:
                print(f'{"No" if num_files == 0 else "More than one"} NIfTI file{"s" if num_files > 1 else ""} found for {series_id}')

                with open('dcm2niix_errors.csv', 'a') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([series_id])

                shutil.rmtree(os.path.join('dcm2niix', series_id))


In [ ]:
check_dcm2niix_errors(f'dcm2niix')

### **Compressing Output Files**


In [ ]:
# Attempt to remove the archive file if it exists
try:
  os.remove('downloadDicomAndConvertNiftiFiles.tar.lz4')
except OSError:
  pass

# Record the start time of the archiving process
start_time = time.time()

# Create a tar archive of the converterType directory, compress it with lz4, and save it as downloadDicomAndConvertNiftiFiles.tar.lz4
!tar cvf - -C {curr_dir} dcm2niix | lz4 > downloadDicomAndConvertNiftiFiles.tar.lz4

# Calculate and record the time taken for the archiving process
archiving_time = time.time() - start_time


### **Utilization Metrics**

In [ ]:
# Save the runtime statistics DataFrame to a CSV file
runtime_stats.to_csv('runtime.csv')

# Add the csv_read_time and archiving_time to the DataFrame as new columns
runtime_stats['archiving_time'] = archiving_time

# Attempt to remove the lz4 file if it exists
try:
  os.remove('downloadDicomAndConvertUsageMetrics.lz4')
except OSError:
  pass

# Compress the runtime.csv file using lz4 and save it as downloadDicomAndConvertUsageMetrics.lz4
!lz4 {curr_dir}/runtime.csv downloadDicomAndConvertUsageMetrics.lz4

# Print the runtime statistics DataFrame
runtime_stats
